In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# 初期化
all_data = []

# データ取得
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=30&smk=r01&po1=25&po2=99&tc=0400301&tc=0400501&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=20.0&md=05&md=06&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&et=9999999&mb=0&mt=9999999&cn=9999999&ta=13&sc=13103&sc=13113&sc=13109&sc=13110&sc=13112"

for page in range(1, 90):
    url = f"{base_url}&page={page}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 各物件の情報取得
    items = soup.find_all("ul", class_="l-cassetteitem")
    for item in items:
        # 物件名クレンジング
        property_name = item.find("div", class_="cassetteitem_content-title").text.strip()
        if not property_name or "駅" in property_name:
            property_name = '物件名登録なし'

        # その他クレンジング
        rent = item.find("span", class_="cassetteitem_other-emphasis ui-text--bold").text.strip().replace('万円', '')
        location = item.find("li", class_="cassetteitem_detail-col1").text.strip()
        walk_to_station = item.find("div", class_="cassetteitem_detail-text").text.strip()
        layout = item.find("span", class_="cassetteitem_madori").text.strip()
        area = item.find("span", class_="cassetteitem_menseki").text.strip().replace('m2', '')
        age_floor_text = item.find("li", class_="cassetteitem_detail-col3").text.strip()
        age, floor = age_floor_text.split('\n', 1)

        # データをリストに追加
        all_data.append({
            "物件名": property_name,
            "家賃": rent,
            "所在地": location,
            "駅徒歩": walk_to_station,
            "間取り": layout,
            "面積": area,
            "築年数": age,
            "階": floor
        })

# データフレームに変換
df = pd.DataFrame(all_data)

# 重複排除
df_unique = df.drop_duplicates(subset=['家賃', '所在地', '間取り', '面積', '築年数', '階'])

# 結果を表示
print(df_unique.head())

# 一意の物件数を確認
print(f"一意の物件数: {len(df_unique)}")


          物件名    家賃         所在地              駅徒歩   間取り     面積   築年数     階
0       ハウス桜丘    13  東京都世田谷区桜丘２   小田急線/千歳船橋駅 歩7分    4K     49  築41年   2階建
1     物件名登録なし  10.5  東京都世田谷区経堂２     小田急線/経堂駅 歩4分    2K  25.46   築3年   4階建
2      ワイズホーム  17.6   東京都目黒区原町１   東急目黒線/西小山駅 歩3分   2DK  66.98  築16年   3階建
3  ロイヤルハウス都立大    18   東京都目黒区平町１  東急東横線/都立大学駅 歩2分  2SDK  55.62  築35年  11階建
5     物件名登録なし    14  東京都世田谷区松原５  京王井の頭線/東松原駅 歩7分    4K  62.42  築50年   2階建
一意の物件数: 509


In [20]:
# 設定
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('green-torus-405215-98138f22cf95.json', scope)
gc = gspread.authorize(credentials)

# スプレッドシートを開く
try:
    spreadsheet = gc.open('Tech0 1130 Homework')
except gspread.SpreadsheetNotFound:
    spreadsheet = gc.create('Tech0 1130 Homework')


# 最初のワークシートを取得
worksheet = spreadsheet.get_worksheet(0)

# データをスプレッドシートに転記
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

/var/folders/ht/sdt7w_3s55d7vk724_f_w8w40000gn/T/ipykernel_2291/1536202176.py:17: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update([df.columns.values.tolist()] + df.values.tolist())


{'spreadsheetId': '1wTUn5QnbbowUn6U1fCgwxVtXXYFejyKglNRifkMUtik',
 'updatedRange': "'シート1'!A1:H533",
 'updatedRows': 533,
 'updatedColumns': 8,
 'updatedCells': 4264}